In [92]:
import os, re, glob  
import cv2  #openCV 라이브러리 import하기
import numpy as np  
from sklearn.model_selection import train_test_split 
from google.colab.patches import cv2_imshow
import cv2

In [8]:
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [113]:
!cp /gdrive/MyDrive/tmp/eyeseyes.zip .
!unzip teameyes.zip

Archive:  teameyes.zip
replace eyes/data/test/close/image10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: eyes/data/test/close/image10.jpg  
replace eyes/data/test/close/image100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: eyes/data/test/close/image100.jpg  
replace eyes/data/test/close/image101.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace eyes/data/test/close/image101.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: eyes/data/test/close/image101.jpg  
  inflating: eyes/data/test/close/image102.jpg  
  inflating: eyes/data/test/close/image103.jpg  
  inflating: eyes/data/test/close/image104.jpg  
  inflating: eyes/data/test/close/image105.jpg  
  inflating: eyes/data/test/close/image106.jpg  
  inflating: eyes/data/test/close/image107.jpg  
  inflating: eyes/data/test/close/image108.jpg  
  inflating: eyes/data/test/close/image109.jpg  
  inflating: eyes/data/test/close/image11.jpg  
  inflating: eyes/data/test/clo

In [150]:
groups_folder_path = '/content/eyes/data/train'
categories = ["close" , "open"]
nb_classes = len(categories) #카테고리갯수: 4개

image_w = 34
image_h = 26


X = []
y = []
for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)] #one-hot으로 라벨을 붙혀줌
    label[idx] = 1

    image_dir = groups_folder_path + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)      #폴더를 열어주고 이미지를 읽음
        img = img.convert("L")   #RGB로 바꿈
         
        
        data = np.asarray(img) #숫자로


        X.append(data) #리스트에 추가 
        y.append(label)


X = np.array(X)
y = np.array(y)
#y = np.argmax(y,axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y) #데이터를 훈련셋과 시험셋으로 나눠주는 함수이용
xy = (X_train, X_test, y_train, y_test)
X_train = X_train.reshape(2391,26,34,1)
y_train = np.argmax(y_train , axis = 1)
y_train = y_train.reshape(2391,1)

X_test = X_test.reshape(797,26,34,1)

y_test = np.argmax(y_test , axis = 1)
y_test = y_test.reshape(797,1)

np.save("multi_image_data.npy", xy) #그렇게 xy를 multi_image_data.npy로 저장

print("ok", len(y))

close  파일 길이 :  990
open  파일 길이 :  2198
ok 3188


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [151]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10, # 0 ~ 180
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=False 
)
iterator = datagen.flow(X_train, y_train, batch_size=1024)

In [152]:
item = next(iterator)
augmented_x, augmented_y = item
print(augmented_x.shape)
print(augmented_y.shape)

(1024, 26, 34, 1)
(1024, 1)


In [149]:

print(y_test.shape)

X_train = X_train.reshape(2391,26,34,1)
print(y_train.shape)
y_train = np.argmax(y_train , axis = 1)
y_train = y_train.reshape(2391,1)


X_test = X_test.reshape(797,26,34,1)
print(y_test.shape)
y_test = np.argmax(y_test , axis = 1)
y_test = y_test.reshape(797,1)

cv2_imshow(X_test[2])
print(y_test[2])
print(X_test[2].shape)
cv2_imshow(X_test[0])

(797, 2)
(2391, 2)
(797, 2)


[1]
(26, 34, 1)


In [155]:
data_count = len(X_train)
print("data_count :", data_count)

WANTED_MULTIPLE_COUNT = 2
print("WANTED_MULTIPLE_COUNT :", WANTED_MULTIPLE_COUNT)

STEP_COUNT = int(data_count/1024)+1
print("STEP_COUNT :", STEP_COUNT)

FETCH_COUNT = STEP_COUNT*WANTED_MULTIPLE_COUNT
print("FETCH_COUNT :", FETCH_COUNT)


augmented_x_list = []
augmented_y_list = []
for i in range(FETCH_COUNT):
  item = next(iterator)
  augmented_x, augmented_y = item
  augmented_x_list.extend(augmented_x)
  augmented_y_list.extend(augmented_y)
  # print(len(augmented_x_list))
# python list를 numpy로 변환
augmented_train_x = np.array(augmented_x_list)
augmented_train_y = np.array(augmented_y_list)

print(augmented_train_x.shape)
print(augmented_train_y.shape)

data_count : 2391
WANTED_MULTIPLE_COUNT : 2
STEP_COUNT : 3
FETCH_COUNT : 6
(4782, 26, 34, 1)
(4782, 1)


In [156]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
plt.style.use('dark_background')

In [157]:
inputs = Input(shape=(26, 34, 1))

net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Flatten()(net)

net = Dense(512)(net)
net = Activation('relu')(net)
net = Dense(1)(net)
outputs = Activation('sigmoid')(net)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 13, 17, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 6, 8, 128)         7385

In [158]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import Callback

In [162]:
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

model.fit(
    augmented_train_x,augmented_train_y, epochs=100 ,validation_data = (X_test , y_test),
    callbacks=[
        ModelCheckpoint('models/%s.h5' % (start_time), monitor='val_acc', save_best_only=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05)
    ]
   
)

#model_check_point = ModelCheckpoint('best_model.h5', monitor='val_loss',  save_best_only=True)

#early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_learning_rate=0.001)
#log_dir = "./logs"


#callbacks = [model_check_point,  early_stopping, reduce_lr]


#history = model.fit(augmented_train_x, augmented_train_y, epochs=300, batch_size=128, verbose=0, validation_split=0.2,
                      #   callbacks=callbacks)

#plt.plot(history.history['loss'], label='train_loss')
#plt.plot(history.history['val_loss'], label='val_loss')
#plt.legend()
#plt.show()

Epoch 1/100
147/150 [============================>.] - ETA: 0s - loss: 0.0089 - acc: 0.9979
Epoch 1: val_acc improved from -inf to 0.99269, saving model to models/2022_07_31_15_54_27.h5
150/150 [==============================] - 1s 8ms/step - loss: 0.0089 - acc: 0.9979 - val_loss: 0.0291 - val_acc: 0.9927 - lr: 5.6295e-38
Epoch 2/100
145/150 [============================>.] - ETA: 0s - loss: 0.0083 - acc: 0.9981
Epoch 2: val_acc did not improve from 0.99269
150/150 [==============================] - 1s 5ms/step - loss: 0.0089 - acc: 0.9979 - val_loss: 0.0291 - val_acc: 0.9927 - lr: 5.6295e-38
Epoch 3/100
149/150 [============================>.] - ETA: 0s - loss: 0.0089 - acc: 0.9979
Epoch 3: val_acc did not improve from 0.99269
150/150 [==============================] - 1s 5ms/step - loss: 0.0089 - acc: 0.9979 - val_loss: 0.0291 - val_acc: 0.9927 - lr: 5.6295e-38
Epoch 4/100
146/150 [============================>.] - ETA: 0s - loss: 0.0082 - acc: 0.9981
Epoch 4: val_acc did not improve

In [163]:
model.save("eyedect_teameye.h5")

In [164]:
!cp eyedect_teameye.h5 /gdrive/MyDrive/tmp

In [169]:
print(model.predict(np.expand_dims(X_test[4], axis = 0)))

cv2_imshow(X_test[4])

[[1.]]
